# HSCT survival: Kullback-Leibler divergence

## Notebook set-up

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler

import configuration as config

notebook_num='02.3'
gpu=0

# Data files
datasets_file=f'{config.PROCESSED_DATA}/01.2-dataset_definitions.pkl'
coxph_survival_file=f'{config.PROCESSED_DATA}/02.1-coxPH_survival.pkl'
weibullaft_survival_file=f'{config.PROCESSED_DATA}/02.2-weibullAFT_survival.pkl'
kld_features_file=f'{config.PROCESSED_DATA}/{notebook_num}-kld_survival.pkl'
kld_models_file=f'{config.MODELS_PATH}/{notebook_num}-kld_models.pkl'

## 1. Load data

In [ ]:
# Read the dataset metadata
with open(datasets_file, 'rb') as input_file:
    datasets=pickle.load(input_file)

# Load one of the datasets
with open(datasets['Continuous target encoded'], 'rb') as input_file:
    data_dict=pickle.load(input_file)

print('Data dictionary contains:\n')
for key, value in data_dict.items():
    print(f' {key}: {type(value)}')

# Load Cox Proportional Hazard model features
with open(coxph_survival_file, 'rb') as input_file:
    coxph_features=pickle.load(input_file)

print('\nCox PH features:\n')
for key, value in coxph_features.items():
    print(f' {key}: {type(value)}')

# Load Weibull Accelerated Failure Time model features
with open(weibullaft_survival_file, 'rb') as input_file:
    weibullaft_features=pickle.load(input_file)

print('\nWeibull AFT features:\n')
for key, value in weibullaft_features.items():
    print(f' {key}: {type(value)}')

## 2. Data preparation

In [ ]:
input_features=['CoxPH survival','CoxPH partial hazard','WeibullAFT survival','WeibullAFT expectation']

training_df=data_dict['Training labels']
training_df['CoxPH survival']=coxph_features['Training survival']
training_df['CoxPH partial hazard']=np.log(coxph_features['Training partial hazard'])
training_df['WeibullAFT survival']=weibullaft_features['Training survival']
training_df['WeibullAFT expectation']=np.log(weibullaft_features['Training expectation'])

testing_df=data_dict['Testing labels']
testing_df['CoxPH survival']=coxph_features['Testing survival']
testing_df['CoxPH partial hazard']=np.log(coxph_features['Testing partial hazard'])
testing_df['WeibullAFT survival']=weibullaft_features['Testing survival']
testing_df['WeibullAFT expectation']=np.log(weibullaft_features['Testing expectation'])

# scaler=StandardScaler()
# scaler.fit(training_df[input_features])
# training_df[input_features]=scaler.transform(training_df[input_features])
# testing_df[input_features]=scaler.transform(testing_df[input_features])

efs_one=training_df[training_df['efs'] == 0]
efs_zero=training_df[training_df['efs'] == 1]

training_df.describe().transpose()

In [ ]:
fig, axs=plt.subplots(2,2, figsize=(8,4))
axs=axs.flatten()

fig.suptitle('Feature distributions')

for i, feature in enumerate(input_features):
    axs[i].hist(efs_one[feature], bins=30, color='firebrick', alpha=0.5, label='EFS 1')
    axs[i].hist(efs_zero[feature], bins=30, color='black', alpha=0.5, label='EFS 0')
    axs[i].set_xlabel(feature)
    axs[i].legend(loc='best')

fig.tight_layout()
fig.show()

## 3. Kernel density estimation

In [ ]:
kdes={}

for feature in input_features:

    feature_min=min(training_df[feature])
    feature_max=max(training_df[feature])
    feature_range=feature_max-feature_min
    padding=feature_range*0.05
    x=np.linspace(feature_min-padding, feature_max+padding)

    efs_zero_kde=gaussian_kde(
        efs_zero[feature].values, 
        bw_method='silverman'
    )

    efs_one_kde=gaussian_kde(
        efs_one[feature].values, 
        bw_method='silverman'
    )

    kdes[feature]={
        'x': x,
        'EFS one': efs_one_kde(x),
        'EFS zero': efs_zero_kde(x)
    }


In [ ]:
fig, axs=plt.subplots(2,2, figsize=(8,4))
axs=axs.flatten()

fig.suptitle('Feature kernel density estimates')

for i, feature in enumerate(input_features):
    axs[i].hist(efs_one[feature], bins=30, density=True, color='firebrick', alpha=0.5, label='EFS 1')
    axs[i].hist(efs_zero[feature], bins=30, density=True, color='black', alpha=0.5, label='EFS 0')
    axs[i].plot(kdes[feature]['x'],kdes[feature]['EFS one'], color='firebrick')
    axs[i].plot(kdes[feature]['x'],kdes[feature]['EFS zero'], color='black')
    axs[i].set_xlabel(feature)
    axs[i].legend(loc='best')

fig.tight_layout()
fig.show()

## 4. Kullback-Leibler divergence

In [ ]:
for feature in input_features:

    # Convert inputs to numpy
    p=np.asarray(kdes[feature]['EFS one'])
    q=np.asarray(kdes[feature]['EFS zero'])

    # Set handling for overflows/underflows - just ignore. We will handle infinite
    # or nan values later by just filtering them out.
    with np.errstate(over='ignore', under='ignore', divide='ignore', invalid='ignore'):
        kld_values=p * np.log2(p/q)

    kdes[feature]['KLD']=kld_values

In [ ]:
fig, axs=plt.subplots(2,2, figsize=(8,6))
axs=axs.flatten()

fig.suptitle('Feature Kullback-Leibler divergence')

for i, feature in enumerate(input_features):
    axs[i].plot(kdes[feature]['x'],kdes[feature]['EFS one'], linestyle='dashed', color='firebrick', label='EFS 1')
    axs[i].plot(kdes[feature]['x'],kdes[feature]['EFS zero'], linestyle='dashed', color='black', label='EFS 0')
    axs[i].plot(kdes[feature]['x'],kdes[feature]['KLD'], color='orange', label='KLD')
    axs[i].set_xlabel(feature)
    axs[i].legend(loc='best')

fig.tight_layout()
fig.show()

## 5. KLD kernel density estimate

In [ ]:
for feature in input_features:

    # Construct new padded x range
    x_min=min(kdes[feature]['x'])
    x_max=max(kdes[feature]['x'])
    x_range=x_max-x_min
    padding=x_range*0.05
    x=np.linspace(x_min-padding, x_max+padding)

    # Shift the kld values so that they are non-negative
    kld_abs=abs(min(kdes[feature]['KLD']))
    kld=kdes[feature]['KLD'] + kld_abs

    # Scale the values so when we convert to integer we get good
    # resolution, e.g. we don't want to collapse 2.1, 2.2, 2.3 etc.,
    # to 2. Instead, 2100.0, 2200.0, 2300.0 become 2100, 2200, 2300 etc.
    kld=kld * 100000

    # Convert to integer
    kld_counts=kld.astype(int)

    # Now, construct a list where each value of x appears a number of times
    # equal to it's KLD 'count'
    kld_scores=[]

    for i, _ in enumerate(kld_counts):
        kld_scores.extend([x[i]] * kld_counts[i])

    kld_kde=gaussian_kde(
        kld_scores, 
        bw_method='silverman'
    )

    # Clip x back to the original range of the data
    feature_min=min(training_df[feature])
    feature_max=max(training_df[feature])
    feature_range=feature_max-feature_min
    padding=feature_range*0.05
    x=np.linspace(feature_min-padding, feature_max+padding)

    kdes[feature]['x']=x
    kdes[feature]['KLD KDE values']=kld_kde(x)
    kdes[feature]['KLD KDE']=kld_kde

In [ ]:
fig, axs=plt.subplots(2,2, figsize=(8,6))
axs=axs.flatten()

fig.suptitle('Feature Kullback-Leibler divergence')

for i, feature in enumerate(input_features):
    axs[i].plot(kdes[feature]['x'],kdes[feature]['EFS one'], linestyle='dashed', color='firebrick', label='EFS 1')
    axs[i].plot(kdes[feature]['x'],kdes[feature]['EFS zero'], linestyle='dashed', color='black', label='EFS 0')
    axs[i].plot(kdes[feature]['x'],kdes[feature]['KLD KDE values'], color='Green', label='KLD KDE')
    axs[i].set_xlabel(feature)
    axs[i].legend(loc='best')

fig.tight_layout()
fig.show()

## 6. Score training and testing data

In [ ]:
for feature in input_features:
    kld_scores=kdes[feature]['KLD KDE'](training_df[feature])
    training_df[f'{feature}_KLD']=kld_scores

    kld_scores=kdes[feature]['KLD KDE'](testing_df[feature])
    testing_df[f'{feature}_KLD']=kld_scores

In [ ]:
fig, axs=plt.subplots(2,2, figsize=(8,6))
axs=axs.flatten()

fig.suptitle('Feature Kullback-Leibler divergence scores')

for i, feature in enumerate(input_features):
    axs[i].scatter(training_df[feature], training_df[f'{feature}_KLD'], s=0.2, color='black')
    axs[i].set_xlabel(feature)
    axs[i].set_ylabel('KLD score')

fig.tight_layout()
fig.show()

## 7. Save KLD features

In [ ]:
training_df.head(len(training_df)).transpose()

In [ ]:
kld_features={
    'Training CoxPH survival KLD':training_df['CoxPH survival_KLD'].values,
    'Training CoxPH partial hazard KLD':training_df['CoxPH partial hazard_KLD'].values,
    'Training WeibullAFT survival KLD':training_df['WeibullAFT survival_KLD'].values,
    'Training WeibullAFT expectation KLD':training_df['WeibullAFT expectation_KLD'].values,
    'Testing CoxPH survival KLD':testing_df['CoxPH survival_KLD'].values,
    'Testing CoxPH partial hazard KLD':testing_df['CoxPH partial hazard_KLD'].values,
    'Testing WeibullAFT survival KLD':testing_df['WeibullAFT survival_KLD'].values,
    'Testing WeibullAFT expectation KLD':testing_df['WeibullAFT expectation_KLD'].values
}

with open(kld_features_file, 'wb') as output_file:
    pickle.dump(kld_features, output_file)

## 8. Save KLD models

In [ ]:
models={}

for feature in input_features:
    models[feature]=kdes[feature]['KLD KDE']

with open(kld_models_file, 'wb') as output_file:
    pickle.dump(models, output_file)